In [ ]:
## 경로 설정
base='2021 빅콘테스트_데이터분석분야_퓨처스리그_홍수ZERO_데이터_210803/'
train_folder='01_제공데이터/'

data=pd.read_excel(base+train_folder+'2021 빅콘테스트_데이터분석분야_퓨처스리그_홍수ZERO_댐유입량,강우,수위데이터_210803.xlsx')

## 열 한글 정리
cols=['홍수사상번호','연','월','일','시간','유입량']

for i in range(1,7):
    prefix=f'데이터집단{i}_'
    cols.append(prefix+'유역평균강수')
    cols.append(prefix+'강우(A지역)')
    cols.append(prefix+'강우(B지역)')
    cols.append(prefix+'강우(C지역)')
    cols.append(prefix+'강우(D지역)')
    cols.append(prefix+'수위(E지역)')
    cols.append(prefix+'수위(B지역)')

## 값 없는 첫행 제거. 
data=data.drop(0).reset_index(drop=True)
data.columns=cols

## 홍수사상번호,연,월,일,시간 정수, 나머지 소수화.
for idx,col in enumerate(cols):
    if idx<5:
        data[col]=data[col].astype(int)
    else:
        data[col]=data[col].astype(float)    
        
## time 변수 생성. 인공신경망에서는 딱히 안쓰지만, lag 변수 병합할 때 사용.
from datetime import datetime,timedelta

data['time']=data.apply(lambda row:datetime(year=int(row['연']),month=int(row['월']),day=int(row['일'])),axis=1)
data['time']=data.apply(lambda row:row['time']+timedelta(hours=row['시간']),axis=1)

## 빅콘테스트 수정사항.
data.loc[2199,'홍수사상번호']=20
data=data.sort_values(by=['홍수사상번호','time'])

## 평가데이터 submission 파일 만들기
test=data.iloc[2891:]

submit=test[['홍수사상번호','연','월','일','시간']]
submit['유입량']=0

plot_data=data.copy()

## 데이터 time 이 있으니 제거.
data=data.drop(['연','월','일','시간'],axis=1)

In [ ]:
## lag 변수 생성.
# lag_1=data.set_index('time').groupby('홍수사상번호')[[col for col in data.columns if '강우' in col]].shift(1).reset_index().fillna(method='bfill')
# lag_2=data.set_index('time').groupby('홍수사상번호')[[col for col in data.columns if '강우' in col]].shift(2).reset_index().fillna(method='bfill')
lag_3=data.set_index('time').groupby('홍수사상번호')[[col for col in data.columns if '강우' in col]].shift(3).reset_index().fillna(method='bfill')
# lag_4=data.set_index('time').groupby('홍수사상번호')[[col for col in data.columns if '강우' in col]].shift(4).reset_index().fillna(method='bfill')

# lag_1.columns=['time']+['lag1_'+col for col in lag_1.columns[1:]]
# lag_2.columns=['time']+['lag2_'+col for col in lag_2.columns[1:]]
lag_3.columns=['time']+['lag3_'+col for col in lag_3.columns[1:]]
# lag_4.columns=['time']+['lag4_'+col for col in lag_4.columns[1:]]

data=pd.merge(data,lag_3,on='time',how='left')
data=data.drop(['time'],axis=1)

In [ ]:
## 전체 데이터를 대상으로 정규화

from sklearn.preprocessing import StandardScaler

for col in data.columns[1:]:
    if col=='유입량':
        mean=data[col].mean()
        std=data[col].std()
        data[col]=(data[col]-mean)/std
        continue
    ss=StandardScaler()
    data[col]=ss.fit_transform(data[col].values.reshape(-1,1))
data.head()

In [ ]:
## train test split
## train을 위해서 일단 나눔.
train=data.iloc[:2865]
test=data.iloc[2865:]

print(f'train.shape : {train.shape},test.shape : {test.shape}')

In [ ]:
# train.shape : (2865, 68),test.shape : (160, 68)

In [ ]:
def window_dataset(df,label=None,window_size=20):
    if label is not None:
        window_list=[]
        label_list=[]
        for i in range(df.shape[0]-window_size):
            window_list.append(np.array(df.iloc[i:i+window_size]))
            label_list.append(np.array(label.iloc[i+window_size]))

        return np.array(window_list),np.array(label_list)
    else:
        window_list=[]
        for i in range(df.shape[0]-window_size):
            window_list.append(np.array(df.iloc[i:i+window_size]))
        return np.array(window_list)

## 홍수사상번호 기준 1~24로 24를 train.
## 홍수사상번호 기준 1~25로 25를 검증.
## 홍수사상번호 기준 1~26로 26을 예측.

# train_x=train[:2694]
# valid_x=train[:2788]

## 홍수사상번호 1빼고
train_x=train[:2668]
valid_x=train[:2762]
test=data.drop(['홍수사상번호','유입량'],axis=1)

train_feature=train_x[data.columns[2:]]
train_label=train_x['유입량']
valid_feature=valid_x[data.columns[2:]]
valid_label=valid_x['유입량']

train_feature, train_label=window_dataset(df=train_feature,label=train_label)
valid_feature, valid_label=window_dataset(df=valid_feature,label=valid_label)
test_feature=window_dataset(df=test)

print(f"train_feature.shape : {train_feature.shape}, train_label.shape : {train_label.shape}")
print(f"valid_feature.shape : {valid_feature.shape}, valid_label.shape : {valid_label.shape}")
print(f"test_feature.shape : {test_feature.shape}")

In [ ]:
# train_feature.shape : (2648, 20, 66), train_label.shape : (2648,)
# valid_feature.shape : (2742, 20, 66), valid_label.shape : (2742,)
# test_feature.shape : (3005, 20, 66)